In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Note spatialsketch ranges also start counting from zero and go up until N-1 (=grid_max)
grid_min = 0
grid_max = 4095

# Make synthetic data in longitude, latitude in range of [grid_min, grid_max]

Properties:

dataset (1):
- uniform distribution over longitude latitude grid
- uniform distribution ip addresses

dataset (2):
- zipf distribution over longitude latitude grid
- uniform distr ip addresses

dataset (3):
- uniform over longitude latitude
- zipf over ip addresses

dataset (4):
- zipf zipf

### Distributions

In [3]:
def get_unif(L, U, seed, N): # Get uniform data within range
    np.random.seed(seed=seed)
    return np.random.randint(L, U, size=N)

In [4]:
def get_zipf(L, U, seed, N): # Get zipf distributed data and cut off points outside of range.
    
    np.random.seed(seed=seed)
    a =1.3
    s = np.random.zipf(a, 2*N)
    s = s[s<U]

    return s

### Dataset creation and output

In [5]:
def create_dataset(ds_id, N):
    df = pd.DataFrame(columns=['timestamp', 'id', 'longitude', 'latitude'])
    ts = np.arange(0, N, 1)
    unif_ids = True
    unif_space = True
    
    if ds_id == 3 or ds_id == 4:
        unif_ids = False
    if ds_id == 2 or ds_id == 4:
        unif_space = False
    if unif_ids:
        ids = get_unif(0, 2147483647,1, N)
    else:
        ids = get_zipf(0, 2147483647, 1, N)
    
    if unif_space:
        longitude = get_unif(grid_min, grid_max,2, N)
        latitude = get_unif(grid_min, grid_max, 3, N)
    else:
        longitude = get_zipf(grid_min, grid_max, 2, N)
        latitude = get_zipf(grid_min, grid_max, 3, N)
    
    df['timestamp'] = pd.Series(ts)
    df['id'] = pd.Series(ids)
    df['longitude'] = pd.Series(longitude)
    df['latitude'] = pd.Series(latitude)

    if (df.isnull().sum().any() > 0):
        print("issue")
        return None
    else:
        write_to_csv(df, ds_id, unif_ids, unif_space, N)
    return df

def write_to_csv(df, ds_id, unif_ids, unif_space, N): # write dataset
    df.to_csv("synth_unifIds{}_unifSpace{}_N{}_grid_res{}-{}.csv".format(unif_ids, unif_space, N, grid_min, grid_max), index=False)
    #print(df, df.dtypes)

### Dataset creation

In [ ]:
def make_datasets(): # Loop over different dataset sizes and settings
    sizes= [40e6]
    for i in [4]:#,2,3,4]:
        for n in sizes:
            create_dataset(i, int(n))
            print("Finished generating size {} for setting {}".format(int(n), i))

make_datasets()